In [1]:
# Filter tensorflow version warnings
import os
# https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import warnings
# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import logging
tf.get_logger().setLevel(logging.ERROR)

In [2]:
import gym
from stable_baselines.common.policies import CnnPolicy #, MlpPolicy, CnnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv 
from stable_baselines import PPO2

from stable_baselines.common.evaluation import evaluate_policy as test
from stable_baselines.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold


In [3]:
## Choose one agent, see Docu for description
agent='CarRacing-v0'
agent='CarRacing-v1'
agent='CarRacing-v2'


In [4]:
## This environment param  
## BEWARE Changing these makes models incompatible!!
game_color = 1
indicators = True
fpst = 4
skip = 3
actions = [[0, 0, 0], [-0.4, 0, 0], [0.4, 0, 0], [0, 1.0, 0], [0, 0, 0.8]]  #this is ACT

In [5]:
## This model param
use = 10       # number of times to use same track [1,100]
ept = 10       # different starting points on same track [1,20]
track_complexity = 12
patience = 2.0
seed = 1000

#using follow_centerline for this first leg of training
REWARD = [-0.0, 0.1, 0.0, 0.0,   1.0, 0.0,   100, -20, -100, -50]

if agent=='CarRacing-v2': 
    env1 = gym.make(agent, seed=seed, 
        game_color=game_color,
        indicators=indicators,
        frames_per_state=fpst,
        skip_frames=skip,
#        discre=actions,       #passing custom actions
        use_track = use,       
        episodes_per_track = ept,  
        tr_complexity = track_complexity, 
        patience = patience,
        off_track = patience,
        f_reward = REWARD )   #passing a custom reward function
else: 
    env1 = gym.make(agent)

env1 = DummyVecEnv([lambda: env1])
env1.metadata

{'render.modes': ['human', 'rgb_array', 'state_pixels'],
 'FPS, 1/timebase': 33.333333333333336,
 'Zoom_level': 1.7,
 'Flight start': False,
 'show_track_1st': False,
 'state_pixels frame size': [96, 96]}

In [6]:
print(env1.action_space)
print(env1.observation_space)

Discrete(5)
Box(0, 255, (96, 96, 4), uint8)


In [7]:
## This training param
batch_size = 256
updates = 500
epochs = 4

# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold = 180, verbose=1)


model = PPO2(CnnPolicy, env1, verbose=1, n_steps=batch_size, # seed=314, n_cpu_tf_sess=1,
             gamma=0.99, learning_rate=0.00025, nminibatches=epochs, ent_coef=0.01, vf_coef=0.5) 


In [8]:
## Separate evaluation env
test_freq = 50      #policy updates until evaluation
test_episodes_per_track = 5   #number of starting points on test_track
eval_log = './evals/'

env_test = gym.make(agent, seed=int(3.14*seed), 
        game_color=game_color,
        indicators=indicators,
        frames_per_state=fpst,
        skip_frames=skip,
        use_track = 1,        #change test track after 1 ept round
        episodes_per_track = test_episodes_per_track,  
        tr_complexity = 12,   #test on a medium complexity track
        patience = 2.0,
        off_track = 2.0,
        discre=actions)       #passing the same custom actions

env_test = DummyVecEnv([lambda: env_test])

eval_callback = EvalCallback(env_test, callback_on_new_best=callback_on_best,  #None,
                             n_eval_episodes=test_episodes_per_track*3, eval_freq=test_freq*batch_size,
                             best_model_save_path=eval_log, log_path=eval_log, deterministic=True, 
                             render=True)  # Turn False not to watch the agent on test, faster


In [9]:
##Independent test routine
#reward_test, epis = test(model, env_test, n_eval_episodes=test_episodes_per_track, 
#                         deterministic=True, render=False, callback=None, reward_threshold=100, 
#                         return_episode_rewards=True)
#reward_test

In [10]:
## Training #1

model.learn(total_timesteps = updates*batch_size, log_interval=1)
#model.learn(total_timesteps = updates*batch_size, log_interval=1, callback=eval_callback)


Track generation: 1226..1540 -> 314-tiles track, complex 12
1  cut by time without progress. Steps 107  %advance 0.9  played reward 9.96  last penalty -20
2  cut by time without progress. Steps 123  %advance 1.5  played reward 8.96  last penalty -20
-------------------------------------
| approxkl           | 0.006985828  |
| clipfrac           | 0.02734375   |
| explained_variance | 0.00678      |
| fps                | 45           |
| n_updates          | 1            |
| policy_entropy     | 1.6032779    |
| policy_loss        | -0.016703583 |
| serial_timesteps   | 256          |
| time_elapsed       | 0            |
| total_timesteps    | 256          |
| value_loss         | 14.011864    |
-------------------------------------
3  cut by time without progress. Steps 153  %advance 1.2  played reward 11.07  last penalty -20
-------------------------------------
| approxkl           | 0.0006264642 |
| clipfrac           | 0.0          |
| explained_variance | 0.015        |
| fps   

11  cut by time without progress. Steps 616  %advance 9.2  played reward 38.92  last penalty -20
-------------------------------------
| approxkl           | 0.009815921  |
| clipfrac           | 0.17773438   |
| explained_variance | 0.256        |
| fps                | 41           |
| n_updates          | 16           |
| policy_entropy     | 1.5599538    |
| policy_loss        | -0.016405117 |
| serial_timesteps   | 4096         |
| time_elapsed       | 94.6         |
| total_timesteps    | 4096         |
| value_loss         | 7.921378     |
-------------------------------------
12  cut by time without progress. Steps 101  %advance 1.5  played reward -0.18  last penalty -20
13  cut by time without progress. Steps 111  %advance 1.2  played reward 5.87  last penalty -20
-------------------------------------
| approxkl           | 0.0034711582 |
| clipfrac           | 0.04296875   |
| explained_variance | 0.214        |
| fps                | 40           |
| n_updates          | 17 

-------------------------------------
| approxkl           | 0.0057453504 |
| clipfrac           | 0.048828125  |
| explained_variance | 0.63         |
| fps                | 40           |
| n_updates          | 30           |
| policy_entropy     | 1.4058906    |
| policy_loss        | -0.003874946 |
| serial_timesteps   | 7680         |
| time_elapsed       | 182          |
| total_timesteps    | 7680         |
| value_loss         | 0.2646995    |
-------------------------------------
27  cut by time without progress. Steps 634  %advance 9.9  played reward 33.92  last penalty -20
--------------------------------------
| approxkl           | 0.0021017906  |
| clipfrac           | 0.0009765625  |
| explained_variance | 0.738         |
| fps                | 41            |
| n_updates          | 31            |
| policy_entropy     | 1.3484144     |
| policy_loss        | 0.00011992478 |
| serial_timesteps   | 7936          |
| time_elapsed       | 188           |
| total_timesteps  

44  cut by time without progress. Steps 192  %advance 1.9  played reward 7.45  last penalty -20
--------------------------------------
| approxkl           | 0.0051989676  |
| clipfrac           | 0.06542969    |
| explained_variance | 0.721         |
| fps                | 40            |
| n_updates          | 44            |
| policy_entropy     | 1.3513836     |
| policy_loss        | -0.0051216753 |
| serial_timesteps   | 11264         |
| time_elapsed       | 271           |
| total_timesteps    | 11264         |
| value_loss         | 3.300218      |
--------------------------------------
45  cut by time without progress. Steps 165  %advance 1.9  played reward 3.33  last penalty -20
46  cut by time without progress. Steps 210  %advance 9.9  played reward 18.85  last penalty -20
-------------------------------------
| approxkl           | 0.011034555  |
| clipfrac           | 0.15039062   |
| explained_variance | 0.897        |
| fps                | 41           |
| n_updates   

--------------------------------------
| approxkl           | 0.009559345   |
| clipfrac           | 0.12109375    |
| explained_variance | 0.588         |
| fps                | 41            |
| n_updates          | 58            |
| policy_entropy     | 1.1340431     |
| policy_loss        | 0.00092778064 |
| serial_timesteps   | 14848         |
| time_elapsed       | 357           |
| total_timesteps    | 14848         |
| value_loss         | 0.7133557     |
--------------------------------------
58  cut by time without progress. Steps 343  %advance 3.1  played reward 29.04  last penalty -20
59  cut by time without progress. Steps 109  %advance 0.9  played reward 8.97  last penalty -20
------------------------------------
| approxkl           | 0.019645613 |
| clipfrac           | 0.4111328   |
| explained_variance | 0.488       |
| fps                | 41          |
| n_updates          | 59          |
| policy_entropy     | 1.094338    |
| policy_loss        | 0.009991458 |
| se

...

20  env max steps reached 1000  %advance 39.9  played reward 108.99  last penalty -50
-------------------------------------
| approxkl           | 0.0026633006 |
| clipfrac           | 0.03515625   |
| explained_variance | -0.00901     |
| fps                | 43           |
| n_updates          | 430          |
| policy_entropy     | 0.74772835   |
| policy_loss        | 0.0016980374 |
| serial_timesteps   | 110080       |
| time_elapsed       | 2.63e+03     |
| total_timesteps    | 110080       |
| value_loss         | 59.066765    |
-------------------------------------
-------------------------------------
| approxkl           | 0.012103912  |
| clipfrac           | 0.12402344   |
| explained_variance | 0.691        |
| fps                | 43           |
| n_updates          | 431          |
| policy_entropy     | 0.79432917   |
| policy_loss        | -0.009198322 |
| serial_timesteps   | 110336       |
| time_elapsed       | 2.64e+03     |
| total_timesteps    | 110336       |
| 

-------------------------------------
| approxkl           | 0.02637011   |
| clipfrac           | 0.2607422    |
| explained_variance | 0.965        |
| fps                | 43           |
| n_updates          | 446          |
| policy_entropy     | 1.030009     |
| policy_loss        | -0.009842873 |
| serial_timesteps   | 114176       |
| time_elapsed       | 2.73e+03     |
| total_timesteps    | 114176       |
| value_loss         | 1.1385821    |
-------------------------------------
26  cut by time without progress. Steps 759  %advance 34.5  played reward 85.54  last penalty -20
--------------------------------------
| approxkl           | 0.006131646   |
| clipfrac           | 0.107421875   |
| explained_variance | 0.935         |
| fps                | 41            |
| n_updates          | 447           |
| policy_entropy     | 0.97669464    |
| policy_loss        | -0.0015611564 |
| serial_timesteps   | 114432        |
| time_elapsed       | 2.74e+03      |
| total_timesteps 

--------------------------------------
| approxkl           | 0.0070094448  |
| clipfrac           | 0.10058594    |
| explained_variance | -0.258        |
| fps                | 43            |
| n_updates          | 462           |
| policy_entropy     | 1.0811222     |
| policy_loss        | -0.0091093015 |
| serial_timesteps   | 118272        |
| time_elapsed       | 2.83e+03      |
| total_timesteps    | 118272        |
| value_loss         | 0.10399829    |
--------------------------------------
-------------------------------------
| approxkl           | 0.010171093  |
| clipfrac           | 0.15722656   |
| explained_variance | -1.17        |
| fps                | 41           |
| n_updates          | 463          |
| policy_entropy     | 1.0537666    |
| policy_loss        | -0.004680859 |
| serial_timesteps   | 118528       |
| time_elapsed       | 2.83e+03     |
| total_timesteps    | 118528       |
| value_loss         | 0.33320934   |
-------------------------------------

--------------------------------------
| approxkl           | 0.008273223   |
| clipfrac           | 0.1328125     |
| explained_variance | -0.833        |
| fps                | 41            |
| n_updates          | 478           |
| policy_entropy     | 1.1194108     |
| policy_loss        | -0.0031179064 |
| serial_timesteps   | 122368        |
| time_elapsed       | 2.92e+03      |
| total_timesteps    | 122368        |
| value_loss         | 0.052983463   |
--------------------------------------
35  cut by time without progress. Steps 804  %advance 44.6  played reward 93.18  last penalty -20
---------------------------------------
| approxkl           | 0.005750738    |
| clipfrac           | 0.076171875    |
| explained_variance | 0.58           |
| fps                | 40             |
| n_updates          | 479            |
| policy_entropy     | 1.0934463      |
| policy_loss        | -0.00092797726 |
| serial_timesteps   | 122624         |
| time_elapsed       | 2.93e+03    

40  cut by time without progress. Steps 221  %advance 8.6  played reward 14.22  last penalty -20
-------------------------------------
| approxkl           | 0.015840836  |
| clipfrac           | 0.21875      |
| explained_variance | 0.391        |
| fps                | 34           |
| n_updates          | 494          |
| policy_entropy     | 1.1841246    |
| policy_loss        | 0.0018688585 |
| serial_timesteps   | 126464       |
| time_elapsed       | 3.02e+03     |
| total_timesteps    | 126464       |
| value_loss         | 18.266056    |
-------------------------------------
41  cut by time without progress. Steps 163  %advance 5.3  played reward 7.71  last penalty -20
-------------------------------------
| approxkl           | 0.0035277782 |
| clipfrac           | 0.0234375    |
| explained_variance | 0.333        |
| fps                | 41           |
| n_updates          | 495          |
| policy_entropy     | 1.1297086    |
| policy_loss        | -0.01522967  |
| serial_

In [11]:
import pickle
root = 'ppo_cnn_gym-mod_'

file = root+'c{:d}_f{:d}_s{:d}_{}_a{:d}__u{:d}_e{:d}_p{}_bs{:d}'.format(
    game_color,fpst,skip,indicators,len(actions),use,ept,patience,batch_size)

model.save(file, cloudpickle=True)
param_list=model.get_parameter_list()


In [12]:
#env1.reset_track()
env1.close()

In [13]:
## This model param  #2
use = 5      # number of times to use same track [1,100]
ept = 8      # different starting points on same track [1,20]
patience = 1.0
seed = 20000
track_complexity = 16   #now training on difficult tracks
REWARD2 = [-0.11, 0.1, 0.0, 0.0,   1.0, 0.0,   100, -20, -100, -50] #adding incentive to move forward fast

if agent=='CarRacing-v2': 
    env2 = gym.make(agent, seed=seed, 
        game_color=game_color,
        indicators=indicators,
        frames_per_state=fpst,
        skip_frames=skip,     
#        discre=actions,       #passing custom actions
        use_track = use,       
        episodes_per_track = ept,  
        tr_complexity = track_complexity, 
        patience = patience,
        f_reward = REWARD2  ) #passing a custom reward function
else: 
    env2 = gym.make(agent)

env2 = DummyVecEnv([lambda: env2])

In [14]:
## Training  #2
new_mod = False  #to change batch_size you need a new model !!
updates = 1000

if new_mod:
    batch_size2 = 512
    model2 = PPO2(CnnPolicy, env2, verbose=1, n_steps=batch_size2, # seed=314, n_cpu_tf_sess=1,
             gamma=0.99, learning_rate=0.00025, nminibatches=epochs, ent_coef=0.01, vf_coef=0.5) 
    model2.load_parameters(param_list, exact_match=True)
    model2.learn(total_timesteps = updates*batch_size2, log_interval=1, callback=eval_callback)
else:
    model.set_env(env2)
    model.learn(total_timesteps = updates*batch_size, log_interval=1, callback=eval_callback)


Track generation: 1391..1741 -> 350-tiles track, complex 16
--------------------------------------
| approxkl           | 0.0079038795  |
| clipfrac           | 0.099609375   |
| explained_variance | -0.211        |
| fps                | 37            |
| n_updates          | 1             |
| policy_entropy     | 1.0963852     |
| policy_loss        | -0.0042322334 |
| serial_timesteps   | 256           |
| time_elapsed       | 0             |
| total_timesteps    | 256           |
| value_loss         | 1.405933      |
--------------------------------------
--------------------------------------
| approxkl           | 0.0043837656  |
| clipfrac           | 0.06640625    |
| explained_variance | 0.644         |
| fps                | 39            |
| n_updates          | 2             |
| policy_entropy     | 1.074409      |
| policy_loss        | -0.0024100137 |
| serial_timesteps   | 512           |
| time_elapsed       | 6.78          |
| total_timesteps    | 512           |
| va

-------------------------------------
| approxkl           | 0.003721349  |
| clipfrac           | 0.048828125  |
| explained_variance | 0.627        |
| fps                | 40           |
| n_updates          | 15           |
| policy_entropy     | 1.1927099    |
| policy_loss        | -0.014012737 |
| serial_timesteps   | 3840         |
| time_elapsed       | 93.4         |
| total_timesteps    | 3840         |
| value_loss         | 6.8599887    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0038716295 |
| clipfrac           | 0.0390625    |
| explained_variance | -0.17        |
| fps                | 38           |
| n_updates          | 16           |
| policy_entropy     | 0.90592736   |
| policy_loss        | -0.005598315 |
| serial_timesteps   | 4096         |
| time_elapsed       | 99.8         |
| total_timesteps    | 4096         |
| value_loss         | 0.23859058   |
-------------------------------------
------------

-------------------------------------
| approxkl           | 0.005165023  |
| clipfrac           | 0.049804688  |
| explained_variance | 0.257        |
| fps                | 38           |
| n_updates          | 31           |
| policy_entropy     | 0.83216363   |
| policy_loss        | -0.009213417 |
| serial_timesteps   | 7936         |
| time_elapsed       | 198          |
| total_timesteps    | 7936         |
| value_loss         | 0.20211063   |
-------------------------------------
18  cut by time without progress. Steps 407  %advance 38.3  played reward 13.83  last penalty -20
-------------------------------------
| approxkl           | 0.01037382   |
| clipfrac           | 0.18066406   |
| explained_variance | 0.615        |
| fps                | 39           |
| n_updates          | 32           |
| policy_entropy     | 1.1135404    |
| policy_loss        | -0.016618513 |
| serial_timesteps   | 8192         |
| time_elapsed       | 205          |
| total_timesteps    | 8192 

27  cut by time without progress. Steps 423  %advance 30.9  played reward 9.19  last penalty -20
--------------------------------------
| approxkl           | 0.0030196984  |
| clipfrac           | 0.040039062   |
| explained_variance | 0.858         |
| fps                | 38            |
| n_updates          | 46            |
| policy_entropy     | 1.0182817     |
| policy_loss        | -0.0025784464 |
| serial_timesteps   | 11776         |
| time_elapsed       | 298           |
| total_timesteps    | 11776         |
| value_loss         | 3.6221972     |
--------------------------------------
28  cut by time without progress. Steps 269  %advance 18.6  played reward 5.29  last penalty -20
29  cut by time without progress. Steps 113  %advance 6.3  played reward 0.05  last penalty -20
-------------------------------------
| approxkl           | 0.0077904295 |
| clipfrac           | 0.09375      |
| explained_variance | 0.871        |
| fps                | 39           |
| n_updates  

-------------------------------------
| approxkl           | 0.003034641  |
| clipfrac           | 0.056640625  |
| explained_variance | -0.12        |
| fps                | 39           |
| n_updates          | 57           |
| policy_entropy     | 0.83142424   |
| policy_loss        | -0.002622691 |
| serial_timesteps   | 14592        |
| time_elapsed       | 433          |
| total_timesteps    | 14592        |
| value_loss         | 0.7086871    |
-------------------------------------
-------------------------------------
| approxkl           | 0.004857888  |
| clipfrac           | 0.055664062  |
| explained_variance | -0.182       |
| fps                | 37           |
| n_updates          | 58           |
| policy_entropy     | 0.9035179    |
| policy_loss        | -0.003646952 |
| serial_timesteps   | 14848        |
| time_elapsed       | 440          |
| total_timesteps    | 14848        |
| value_loss         | 0.59049296   |
-------------------------------------
------------

...

------------------------------------
| approxkl           | 0.007641207 |
| clipfrac           | 0.083984375 |
| explained_variance | 0.701       |
| fps                | 42          |
| n_updates          | 933         |
| policy_entropy     | 0.61096823  |
| policy_loss        | 0.003343213 |
| serial_timesteps   | 238848      |
| time_elapsed       | 7.81e+03    |
| total_timesteps    | 238848      |
| value_loss         | 1.8604497   |
------------------------------------
3  cut by time without progress. Steps 364  %advance 48.1  played reward 27.25  last penalty -20
-------------------------------------
| approxkl           | 0.0077003227 |
| clipfrac           | 0.072265625  |
| explained_variance | 0.795        |
| fps                | 42           |
| n_updates          | 934          |
| policy_entropy     | 0.5572649    |
| policy_loss        | -0.008751893 |
| serial_timesteps   | 239104       |
| time_elapsed       | 7.82e+03     |
| total_timesteps    | 239104       |
| va

-------------------------------------
| approxkl           | 0.006581408  |
| clipfrac           | 0.095703125  |
| explained_variance | -0.0493      |
| fps                | 38           |
| n_updates          | 949          |
| policy_entropy     | 0.65621495   |
| policy_loss        | -0.010630509 |
| serial_timesteps   | 242944       |
| time_elapsed       | 7.91e+03     |
| total_timesteps    | 242944       |
| value_loss         | 0.018223226  |
-------------------------------------
1  Finalized in Steps 606  with return=total_reward 138.3888888888886
2  Finalized in Steps 606  with return=total_reward 138.3888888888886
3  Finalized in Steps 630  with return=total_reward 135.9888888888886
4  Finalized in Steps 596  with return=total_reward 139.38888888888863
5  Finalized in Steps 604  with return=total_reward 138.58888888888862
Track generation: 1187..1487 -> 300-tiles track, complex 12
1  Finalized in Steps 660  with return=total_reward 133.09665551839427
2  Finalized in Steps 6

-------------------------------------
| approxkl           | 0.0060126167 |
| clipfrac           | 0.095703125  |
| explained_variance | -0.585       |
| fps                | 38           |
| n_updates          | 963          |
| policy_entropy     | 0.59434736   |
| policy_loss        | -0.006649752 |
| serial_timesteps   | 246528       |
| time_elapsed       | 8.15e+03     |
| total_timesteps    | 246528       |
| value_loss         | 0.040594712  |
-------------------------------------
-------------------------------------
| approxkl           | 0.008267656  |
| clipfrac           | 0.1015625    |
| explained_variance | 0.607        |
| fps                | 41           |
| n_updates          | 964          |
| policy_entropy     | 0.65774244   |
| policy_loss        | -0.009428497 |
| serial_timesteps   | 246784       |
| time_elapsed       | 8.16e+03     |
| total_timesteps    | 246784       |
| value_loss         | 0.021962423  |
-------------------------------------
14  Finalize

19  Finalized in Steps 718  with return=total_reward 173.37029046587995
--------------------------------------
| approxkl           | 0.0061029242  |
| clipfrac           | 0.0859375     |
| explained_variance | 0.0748        |
| fps                | 41            |
| n_updates          | 979           |
| policy_entropy     | 0.3959881     |
| policy_loss        | -0.0010101837 |
| serial_timesteps   | 250624        |
| time_elapsed       | 8.25e+03      |
| total_timesteps    | 250624        |
| value_loss         | 125.00241     |
--------------------------------------
--------------------------------------
| approxkl           | 0.012515646   |
| clipfrac           | 0.078125      |
| explained_variance | 0.429         |
| fps                | 41            |
| n_updates          | 980           |
| policy_entropy     | 0.42554224    |
| policy_loss        | -0.0075946236 |
| serial_timesteps   | 250880        |
| time_elapsed       | 8.26e+03      |
| total_timesteps    | 250880  

--------------------------------------
| approxkl           | 0.009647422   |
| clipfrac           | 0.09472656    |
| explained_variance | 0.371         |
| fps                | 42            |
| n_updates          | 995           |
| policy_entropy     | 0.5530812     |
| policy_loss        | -0.0011547611 |
| serial_timesteps   | 254720        |
| time_elapsed       | 8.35e+03      |
| total_timesteps    | 254720        |
| value_loss         | 0.0709304     |
--------------------------------------
25  Finalized in Steps 701  with return=total_reward 174.52650571920242
------------------------------------
| approxkl           | 0.01209136  |
| clipfrac           | 0.17089844  |
| explained_variance | 0.122       |
| fps                | 42          |
| n_updates          | 996         |
| policy_entropy     | 0.6407254   |
| policy_loss        | 0.002174265 |
| serial_timesteps   | 254976      |
| time_elapsed       | 8.35e+03    |
| total_timesteps    | 254976      |
| value_loss  

In [15]:
#import pickle
if new_mod:
    file = root+'c{:d}_f{:d}_s{:d}_{}_a{:d}__u{:d}_e{:d}_p{}_bs{:d}'.format(
        game_color,fpst,skip,indicators,len(actions),use,ept,patience,batch_size2)
    model2.save(file, cloudpickle=True)
    param_list=model2.get_parameter_list()
else:
    model.save(file+'II', cloudpickle=True)
    param_list=model.get_parameter_list()


In [16]:
env2.close()
env_test.close()

In [27]:
## Enjoy last trained policy

if agent=='CarRacing-v2':  #create an independent test environment
    env3 = gym.make(agent, seed=None, 
        game_color=game_color,
        indicators = True,
        frames_per_state = fpst,
        skip_frames = skip,   
#        discre = actions,
        use_track = 2,       
        episodes_per_track = 1,  
        tr_complexity = 12, 
        patience = 5.0,
        off-track = 5.0 )
else:
    env3 = gym.make(agent)

env3 = DummyVecEnv([lambda: env3])
obs = env3.reset()
print(obs.shape)        

done = False
pasos = 0
_states=None

while not done and pasos<1002:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env3.step(action)
    env3.render()
    pasos+=1
    
env3.close()
print()
print(reward, done, pasos, info)

Track generation: 1164..1458 -> 294-tiles track, complex 12
(1, 96, 96, 4)
1  Finalized in Steps 644  with return=total_reward 134.67610921501688

[100.] [ True] 644 [{'terminal_observation': array([[[176.549, 176.549, 161.874, 161.874],
        [176.549, 176.549, 161.874, 161.874],
        [176.549, 176.549, 161.874, 161.874],
        ...,
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874]],

       [[176.549, 176.549, 161.874, 161.874],
        [176.549, 176.549, 161.874, 161.874],
        [176.549, 176.549, 161.874, 161.874],
        ...,
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874]],

       [[176.549, 176.549, 161.874, 161.874],
        [176.549, 176.549, 161.874, 161.874],
        [176.549, 176.549, 161.874, 161.874],
        ...,
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 16

In [28]:
## Enjoy best eval_policy

obs = env3.reset()
print(obs.shape)        

## Load bestmodel from eval
#if not isinstance(model_test, PPO2):
model_test = PPO2.load(eval_log+'best_model', env3)

done = False
pasos = 0
_states=None

while not done and pasos<1002:
    action, _states = model_test.predict(obs, deterministic=True)
    obs, reward, done, info = env3.step(action)
    env3.render()
    pasos+=1
    
env3.close()
print()
print(reward, done, pasos)
print(action, _states)

Track generation: 1084..1358 -> 274-tiles track, complex 12
(1, 96, 96, 4)
0  Finalized in Steps 544  with return=total_reward 144.60109890109854

[100.] [ True] 544
[3] None


In [19]:
model_test.save(file+'_evalbest', cloudpickle=True)

In [20]:
env2.close()

In [21]:
env3.close()

In [22]:
env_test.close()

In [23]:
print(action, _states)

[0] None


In [24]:
obs.shape

(1, 96, 96, 4)